# Исследование надёжности заёмщиков

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение 10 строк таблицы df
df.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6615,0,-8922.173305,58,среднее,1,женат / замужем,0,F,сотрудник,0,165067.010952,покупка недвижимости
18998,0,350871.171048,56,среднее,1,женат / замужем,0,F,пенсионер,0,204362.073077,операции со своей недвижимостью
695,0,364697.666024,57,среднее,1,женат / замужем,0,F,пенсионер,0,114356.625624,на покупку подержанного автомобиля
6374,0,-825.644363,28,среднее,1,гражданский брак,1,M,сотрудник,0,146125.146776,на покупку автомобиля
19886,0,383504.795288,54,среднее,1,женат / замужем,0,F,пенсионер,0,244851.508381,покупка коммерческой недвижимости
1749,1,-658.506604,24,Среднее,1,Не женат / не замужем,4,M,сотрудник,0,132982.952552,жилье
14383,0,-6704.354083,55,среднее,1,в разводе,3,M,сотрудник,0,192189.509297,операции с недвижимостью
10473,1,-879.610066,40,среднее,1,женат / замужем,0,M,сотрудник,0,139322.554708,получение дополнительного образования
3145,0,333008.343538,56,среднее,1,в разводе,3,M,пенсионер,0,65350.861129,сделка с автомобилем
4522,0,-4092.078716,49,Высшее,0,женат / замужем,0,F,сотрудник,0,125056.753309,профильное образование


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Таблица состоит из 12 столбцов, из которых столбцы `days_employed` и `total_income` с типом данных `float64`, пять столбцов (`children`, `dob_years`, `education_id`, `family_status_id`, `debt`) с типом данных `int64`, а оставшиеся пять столбцов (`education`, `family_status`, `gender`, `income_type`, `purpose`) - `object`.
В каждой строке таблицы — данные о клиентах банка. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

Не все пропущенные значения влияют на исследование. В столбце `days_employed` пропуски не важны для данной работы. Количество пропусков в столбцах `days_employed` и `total_income` равно, можно предположить, что либо пропуски оставленны умышленно, либо система сбора информации дала сбой/некорректная выгрузка данных.

Тем не менее, пропуски нельзя оставлять без внимания: группировка данных с `NaN` может привести к некорректным результатам анализа. Трудовой стаж и доход — это это количественные переменные. Разделим данные по группам по типу занятости и заполним пропуски медианными значениями.

In [6]:
# преобразование значений столбца days_employed в абсолютные значения
df['days_employed'] = df['days_employed'].abs()

In [7]:
# группировка данных по типу занятости и просмотр медианных значений столбца days_employed
df_grouped = df.groupby('income_type')['days_employed'].median()
df_grouped

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [8]:
# замена стажа безработных и пенсионеров на стаж, деленный на 24 
df.loc[df['income_type']=='безработный', 'days_employed'] = df.loc[df['income_type']=='безработный', 'days_employed']/24
df.loc[df['income_type']=='пенсионер', 'days_employed'] = df.loc[df['income_type']=='пенсионер', 'days_employed']/24

In [9]:
# проверка
df.groupby('income_type')['days_employed'].median()

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

In [10]:
#замена пропущенных значений в стролбце days_employed на медианные значения
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.median()))

In [11]:
# группировка данных по типу занятости и подсчет медианных значений столбца total_income
df_grouped_income = df.groupby('income_type')['total_income'].median()
df_grouped_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [12]:
#замена пропущенных значений в стролбце total_income на медианные значения
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [13]:
# проверка отсутствия пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замена типа данных

Чтобы превести вещественные числа в целые воспользуемся методом `astype()`

In [14]:
# замена вещественного типа данных в столбцах days_employed и total_income на целочисленный
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [15]:
# проверка типов данных
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Мы заменили в двух столбцах вещественный тип данных и теперь наши данные имеют только два типа: целочисленный и строковый.

### Обработка дубликатов

Чаще всего дубликаты возникают в результате ошибок пользователя при занесении информации, неправильном соединение данных из разных источников или сбое при выгрузке данных.

Для подсчета количества дубликатов воспользуемся методом `duplicated()`, который в сочетании с методом `sum()` возвращает количество дубликатов, а для удаления дубликатов вызовем метод `drop_duplicates()`

In [16]:
# просмотр уникальных значений количества детей
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [17]:
# замена отрицптельного значения на 0
df.loc[df['children']==-1, 'children'] = 0

In [18]:
# проверка
df['children'].value_counts()

0     14196
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [19]:
# просмотр уникальных значений столбца dob_years
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [20]:
# группировка данных по типу занятости и подсчет медианных значений столбца dob_years
df.groupby('income_type')['dob_years'].median()

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [21]:
# создание функции для подсчета медианных значений для каждого типа занятия
def dob_years_median(income):
    years_median = df[df['income_type']==income]['dob_years'].median()
    return years_median

In [22]:
# проверка работы функции
dob_years_median('пенсионер')

60.0

In [23]:
# замена нулевых значений медианными
df.loc[df['dob_years']==0, 'dob_years'] = df.loc[df['dob_years']==0, 'income_type'].apply(dob_years_median)

In [24]:
# проверка
(df['dob_years']==0).value_counts()

False    21525
Name: dob_years, dtype: int64

In [25]:
# просмотр списка уникальных значений столбца education
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [26]:
# замена значений столбца education на строчные
df['education'] = df['education'].str.lower()

In [27]:
# проверка
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [28]:
# подсчёт явных дубликатов
df.duplicated().sum()

71

In [29]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

In [30]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

**Вывод**

Было обнаружено и удалено 71 явных дубликата, без них исследование станет более точным. Кроме того были найдены неявные дубликаты уровней образования, а также исправлены ошибки в данных.

### Лемматизация

In [31]:
# импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()

In [32]:
def lemmas_result(row): # создаем функцию, обозначим строку переменной row
    purpose = row['purpose'] #обращаемся к конкретным значениям столбца row['purpose']
    lemmas = ' '.join(m.lemmatize(purpose)) # лемматизируем значения столбца purpose и склеиваем результат вызовом метода join()
    return lemmas
df['lemmas'] = df.apply(lemmas_result, axis=1) # создаем новый столбец 

In [33]:
# просмотр первых 10 строк таблицы df с результатом лемматизации
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n


**Вывод**

Для дальнейшей категоризации данных, были выделины леммы в значениях столбца с целями получения кредита. Теперь можно приступить к категоризации целей

### Категоризация данных

In [34]:
# импорт модуля Counter
from collections import Counter

In [35]:
# объединяем в строку цели из столбца purpose, лематизируем и подсчитываем число упоминаний слов с помощью Counter
Counter(m.lemmatize(' '.join(df['purpose'])))

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

In [36]:
# создание функции для объединения целей кредита в категории
def purpose_group(lemmas): 
    if ('жилье' in lemmas) | ('недвижимость' in lemmas):
        return 'покупка жилья/недвижимости'
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    return 'другое'

In [37]:
# тестируем работу функции для каждого правила
print(purpose_group('покупка   свой   жилье'))
print(purpose_group('строительство   жилой   недвижимость'))
print(purpose_group('профильный   образование'))
print(purpose_group('сделка   с   подержанный   автомобиль'))
print(purpose_group('сыграть   свадьба'))

покупка жилья/недвижимости
покупка жилья/недвижимости
образование
покупка автомобиля
свадьба


In [38]:
# создание отдельного столбеца с категориями целей кредита
df['purpose_group'] = df['lemmas'].apply(purpose_group)
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,покупка жилья/недвижимости
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,покупка автомобиля
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,покупка жилья/недвижимости
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,покупка жилья/недвижимости
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,покупка жилья/недвижимости
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,покупка жилья/недвижимости


In [39]:
# вызов метода qcut(), чтобы посмотреть на какие группы по уровню дохода можно разделить данные
pd.qcut(df['total_income'], 6).value_counts()

(119261.0, 142594.0]     3691
(142594.0, 172357.0]     3594
(228876.5, 2265604.0]    3576
(92146.5, 119261.0]      3576
(20666.999, 92146.5]     3576
(172357.0, 228876.5]     3441
Name: total_income, dtype: int64

In [40]:
# создание функции для разделения данных на группы одинакового размера по уровню дохода
def total_income_group(total_income):
    #total_income = row['total_income']
    if total_income<=92182:
        return '20666.0 - 92183.0'
    if total_income<=119253:
        return '92183.0 - 119254.0'
    if total_income<=142594:
        return '119254.0 - 142594.0'
    if total_income<=172357:
        return '142594.0 - 172357.0'
    if total_income<=228838:
        return '172357.0 - 228838.0'
    return '228838.0 - 2265604.0'

In [41]:
# тестируем работу функции
total_income_group(50000)

'20666.0 - 92183.0'

In [42]:
# создание отдельного столбеца с категориями доходов
df['total_income_group'] = df['total_income'].apply(total_income_group)
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_group,total_income_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,покупка жилья/недвижимости,228838.0 - 2265604.0
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,покупка автомобиля,92183.0 - 119254.0
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,покупка жилья/недвижимости,142594.0 - 172357.0
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,228838.0 - 2265604.0
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,142594.0 - 172357.0
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,покупка жилья/недвижимости,228838.0 - 2265604.0
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,покупка жилья/недвижимости,228838.0 - 2265604.0
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование,119254.0 - 142594.0
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба,92183.0 - 119254.0
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,покупка жилья/недвижимости,142594.0 - 172357.0


**Вывод**

Таким образом, было выделено 4 категории целей: покупка жилья/недвижимости, образование, покупка автомобиля и свадьба. Так как покупка жилья и покупка недвижимости по смыслу одно и тоже, цели про жилье и недвижимость были объеденены в одну. Теперь данные подготовлены, чтобы делать из них выводы

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
# создание сводной таблицы по количеству детей 
df.pivot_table(index =['children'], values='debt', aggfunc='mean')

,debt
children,
0,0.075258
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


**Вывод**

Зависимость между наличием детей и возвратом кредита в срок невыявлена. Процент невозврата кредита в срок в каждой группе примерно одинаковый, около 0,1 или 10%. Наибольший процент невозврата кредита составил 10,53% в категории клиентов, у которых свыше 5 детей. У клиентов, количество детей у которых 5, процент равен нулю, что все кредиты были возвращены вовремя.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
# создание сводной таблицы по семейному положению
df.pivot_table(index =['family_status'], values='debt', aggfunc='mean')

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок невыявлена. Процент возврата кредита в срок в каждой группе примерно одинаковый. При этом, можно отметить, что наименьший процент невозврата кредита в срок у категорий клиентов, кто  является вдовцом (вдовой), а наибольший процент у катекорий, что живет в гражданском браке или вообще не женат (не замужем)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
# группировка значений по уровню дохода и возврату кредита
df.groupby('total_income_group')['debt'].value_counts(normalize=True)

total_income_group    debt
119254.0 - 142594.0   0       0.910937
                      1       0.089063
142594.0 - 172357.0   0       0.915693
                      1       0.084307
172357.0 - 228838.0   0       0.919430
                      1       0.080570
20666.0 - 92183.0     0       0.920089
                      1       0.079911
228838.0 - 2265604.0  0       0.929869
                      1       0.070131
92183.0 - 119254.0    0       0.917367
                      1       0.082633
Name: debt, dtype: float64

**Вывод**

Зависимость между целью кредита и возвратом кредита в срок невыявлена. Данные были разделены на 6 категорий уровня дохода. Процент возврата кредита в срок в каждой так же примерно одинаковый. Наибольший процент (92%) возврата составил у двух групп с доходом от 20 до 92 тысяч в месяц и от 228 до 2265 тысяч в месяц. Остальные категории имеют 91% возвратов кредитов в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
# группировка значений по цели кредита и возврату кредита
df.groupby('purpose_group')['debt'].value_counts(normalize=True)

purpose_group               debt
образование                 0       0.907800
                            1       0.092200
покупка автомобиля          0       0.906410
                            1       0.093590
покупка жилья/недвижимости  0       0.927666
                            1       0.072334
свадьба                     0       0.919966
                            1       0.080034
Name: debt, dtype: float64

**Вывод**

Зависимость между целью кредита и возвратом кредита в срок невыявлена. Процент возврата кредита в срок в каждой группе примерно одинаковый. 92% клиентов, взявших кредит на покупку жилья (недвижимости) или на свадьбу, вернули кредит в срок. А у клиентов, целью кредита которых было оьбразование или покупка втомобиля, это значение составило 90%.

## Шаг 4. Общий вывод

Было проведено исследования влияния 4 признаков на факт погашения кредита в срок: количества детей, семейного положения, уровеня дохода и целей кредита. В ходе исследования были обнаружены ошибки, дубликаты и пропуски в данных. Проблемы в данных могли возникнуть по нескольким причинам: была произведена некорректная выгрузка данных или система сбора информации дала сбой, пропуски и ошибки могли быть оставленны умышленно/случайно самими клиентами банка.Чтобы это не повлияло на результат исследования, была проведена предобработка данных.

Было удалено 54 дубликата, а также исправлено альтернативное написание значений, дабы убрать неявниые дубликаты, пропущенные значения были заменены на медианные для каждой группы соответственно.

Чтобы ответить на вопрос, как влияют цели кредита на возврат в срок, была проведена лемматизация целей, в результате которой, было выделено 4 категории целей: покупка жилья/недвижимости, образование, покупка автомобиля и свадьба.

Рассматривая влияние каждого признака на возврат кредита, были получены следующие результаты: процент возврата по кредиту в каждой группе примерно одинаковый и колеблется в районе 90%. То есть не было выявлено зависимости между признаками и возвратом кредита в срок.